<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/SMA_Model_Backtest_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninst

In [1]:
import sys
import os

# Point to your Drive folder
drive_lib = '/content/drive/My Drive/python_libs'
if drive_lib not in sys.path:
    sys.path.insert(0, drive_lib)

try:
    import pandas_ta as ta
    import pandas as pd
    import numpy as np
    print("Success! Environment is ready for 2022 analysis.")
    print(f"Pandas Version: {pd.__version__}")
except Exception as e:
    print(f"Error: {e}")

Success! Environment is ready for 2022 analysis.
Pandas Version: 3.0.0


In [2]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np

# 1. Configuration
SMA_RANGE = range(3, 21)
BB_PERIODS = [10, 20, 30]
SYMBOLS = ["TSLA", "SPY", "QQQ", "NVDA", "META", "MSTR", "COIN", "GLD", "AMD", "SLV", "PLTR", "MSFT", "ORCL", "IWM", "AAPL", "AVGO", "AMZN", "UNH", "NFLX", "MU", "GOOGL", "TSM", "LULU", "CRWV", "GOOG", "IBIT", "JPM", "HOOD", "GDX", "ADBE", "NOW", "APP", "GS", "WOLF", "BABA", "IREN", "COST", "INTC", "LLY", "CRCL", "CVNA", "SNDK", "OKLO", "SMH", "BA", "BMNR", "ASTS", "NBIS", "SOFI", "BE"]

# 2. Optimized Data Cache
print("Fetching fresh data...")
data_cache = {}
for s in SYMBOLS:
    df = yf.download(s, period="5y", interval="1d", progress=False, auto_adjust=True)
    #df = yf.download(s, start="2021-12-01", end="2023-01-01", interval="1d", progress=False, auto_adjust=True)
    if not df.empty:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        data_cache[s] = df

# 3. Generate Raw Trades with Technical Audit Columns
all_trades_list = []
print("Generating Technical Audit Logs...")

for sma_val in SMA_RANGE:
    for bb_val in BB_PERIODS:
        for symbol, df_orig in data_cache.items():
            df = df_orig.copy()

            # Indicators
            df['SMA_Value'] = ta.sma(df['Close'], length=sma_val)
            bb = ta.bbands(df['Close'], length=bb_val, std=2)
            if bb is None: continue

            bbm_col = [c for c in bb.columns if c.startswith('BBM')][0]
            df['BB_Midpoint'] = bb[bbm_col]

            # Logic: Cross above SMA AND Below BB Midpoint
            df['Prev_Close'] = df['Close'].shift(1)
            df['Prev_SMA'] = df['SMA_Value'].shift(1)

            cond_cross = (df['Close'] > df['SMA_Value']) & (df['Prev_Close'] < df['Prev_SMA'])
            cond_value = df['Close'] < df['BB_Midpoint']

            # Filter and Label
            trades = df[cond_cross & cond_value].copy()
            if not trades.empty:
                # Forward Returns
                trades['Ret_3D'] = df['Close'].pct_change(3).shift(-3)
                trades['Ret_5D'] = df['Close'].pct_change(5).shift(-5)
                trades['Ret_10D'] = df['Close'].pct_change(10).shift(-10)

                # Metadata
                trades['Symbol'] = symbol
                trades['SMA_Param'] = sma_val
                trades['BB_Param'] = bb_val

                # Selecting Audit Columns
                audit_cols = [
                    'Symbol', 'SMA_Param', 'BB_Param',
                    'Close', 'SMA_Value', 'BB_Midpoint',
                    'Ret_3D', 'Ret_5D', 'Ret_10D'
                ]
                all_trades_list.append(trades[audit_cols])

master_trade_log = pd.concat(all_trades_list)
master_trade_log = master_trade_log.reset_index() # Keep the Date column
print(f"Done! {len(master_trade_log)} total trade instances recorded.")

# VIEW SPECIFIC SYMBOL DETAIL (Example: NVDA)
display(master_trade_log[master_trade_log['Symbol'] == 'NVDA'].head(20))

Fetching fresh data...
Generating Technical Audit Logs...
Done! 86939 total trade instances recorded.


Price,Date,Symbol,SMA_Param,BB_Param,Close,SMA_Value,BB_Midpoint,Ret_3D,Ret_5D,Ret_10D
257,2021-03-01,NVDA,3,10,13.802934,13.583051,14.344884,-0.106309,-0.162443,-0.046667
258,2021-03-09,NVDA,3,10,12.489450,12.158915,13.015729,0.026817,0.061580,0.043969
259,2021-03-26,NVDA,3,10,12.807665,12.641324,12.934750,0.039644,0.089433,0.184571
260,2021-04-23,NVDA,3,10,15.227696,15.121375,15.384484,0.000753,-0.016754,-0.029675
261,2021-05-06,NVDA,3,10,14.487272,14.408716,14.953672,-0.014925,-0.059061,0.006163
262,2021-05-14,NVDA,3,10,14.207962,13.854749,14.286817,-0.012445,0.052570,0.140525
263,2021-07-19,NVDA,3,10,18.737843,18.594080,19.709345,0.043358,0.027383,0.051665
264,2021-08-12,NVDA,3,10,19.860582,19.802380,19.972931,-0.022457,-0.005376,0.108666
265,2021-08-19,NVDA,3,10,19.753820,19.388638,19.819175,0.100768,0.114658,0.131425
266,2021-09-10,NVDA,3,10,22.431799,22.285432,22.457444,-0.006095,-0.025714,-0.017662


In [3]:
# 1. Grouping by Symbol and Strategy Parameters
performance_summary = master_trade_log.groupby(['Symbol', 'SMA_Param', 'BB_Param']).agg(
    Trade_Count=('Ret_3D', 'count'),
    Wins_3D=('Ret_3D', lambda x: (x > 0).sum()),
    Wins_5D=('Ret_5D', lambda x: (x > 0).sum()),
    Wins_10D=('Ret_10D', lambda x: (x > 0).sum()),
    Avg_3D_Ret=('Ret_3D', 'mean'),
    Avg_5D_Ret=('Ret_5D', 'mean'),
    Avg_10D_Ret=('Ret_10D', 'mean')
).reset_index()

# 2. Calculate Win Rates
performance_summary['Win_Rate_3D'] = performance_summary['Wins_3D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_5D'] = performance_summary['Wins_5D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_10D'] = performance_summary['Wins_10D'] / performance_summary['Trade_Count']

# Define display columns for consistency
display_cols = [
    'Symbol', 'SMA_Param', 'BB_Param', 'Trade_Count',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

# --- SECTION A: Best Parameter Set PER Symbol ---
# Sort by Symbol and Win Rate, then take the top 1 for each ticker
best_per_symbol = performance_summary.sort_values(
    ['Symbol', 'Win_Rate_10D', 'Avg_10D_Ret'],
    ascending=[True, False, False]
).groupby('Symbol').head(1)

print("--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---")
display(best_per_symbol[display_cols].sort_values('Win_Rate_10D', ascending=False).head(25).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))


--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
308,ASTS,19,20,3,100.0%,10.23%,100.0%,12.94%,100.0%,12.54%
188,AMZN,14,10,8,37.5%,0.95%,87.5%,2.13%,100.0%,4.26%
764,CRWV,18,20,1,100.0%,2.82%,100.0%,12.24%,100.0%,20.67%
807,CVNA,16,10,6,100.0%,25.98%,83.3%,23.02%,100.0%,27.56%
387,BA,11,10,1,100.0%,0.24%,100.0%,9.43%,100.0%,8.31%
560,BMNR,17,10,1,100.0%,6.16%,100.0%,18.79%,100.0%,6.82%
702,CRCL,13,10,1,100.0%,7.75%,100.0%,4.23%,100.0%,21.12%
2370,SPY,19,20,2,50.0%,1.25%,100.0%,3.43%,100.0%,5.05%
2293,SOFI,11,10,2,50.0%,0.95%,50.0%,13.11%,100.0%,13.19%
2268,SNDK,19,30,2,100.0%,7.70%,100.0%,13.21%,100.0%,24.64%


In [4]:
# 1. Global Aggregation: Using the Raw Trade Log
# This avoids 'average of averages' for both Win Rates AND Returns.
global_strategy_raw = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Symbol': 'count',      # Total Trade Count
    'Ret_3D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_5D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_10D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ]
})

# 2. Flatten Multi-index columns for cleaner access
global_strategy_raw.columns = [
    'Total_Trades',
    'Wins_3D', 'Avg_3D_Ret',
    'Wins_5D', 'Avg_5D_Ret',
    'Wins_10D', 'Avg_10D_Ret'
]
global_strategy_raw = global_strategy_raw.reset_index()

# 3. Recalculate Global Win Rates from raw counts
global_strategy_raw['Win_Rate_3D'] = global_strategy_raw['Wins_3D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_5D'] = global_strategy_raw['Wins_5D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_10D'] = global_strategy_raw['Wins_10D'] / global_strategy_raw['Total_Trades']

# 4. Final Formatting & Sorting
final_cols = [
    'SMA_Param', 'BB_Param', 'Total_Trades',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

global_strategy_raw = global_strategy_raw[final_cols].sort_values(by='Win_Rate_10D', ascending=False)

print("--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---")
display(global_strategy_raw.head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---


,SMA_Param,BB_Param,Total_Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
26,12,10,307,50.2%,0.32%,55.0%,0.89%,58.6%,1.71%
32,14,10,398,53.0%,0.80%,54.3%,0.97%,57.3%,2.15%
23,11,10,200,52.5%,0.94%,51.5%,0.77%,56.5%,0.82%
27,12,20,1481,53.2%,0.48%,53.7%,0.50%,56.0%,1.17%
24,11,20,1649,53.2%,0.37%,52.9%,0.33%,56.0%,1.29%
25,11,30,1839,55.4%,0.48%,54.1%,0.47%,56.0%,1.44%
29,13,10,372,51.1%,0.39%,53.0%,0.75%,55.9%,1.49%
28,12,30,1714,55.4%,0.51%,54.8%,0.56%,55.9%,1.32%
33,14,20,1188,52.7%,0.53%,52.6%,0.55%,55.5%,1.24%
30,13,20,1335,52.4%,0.42%,53.0%,0.39%,55.1%,1.08%


In [5]:
# --- Symbol Drill-Down Section ---
target_symbol = "NVDA"  # Change this to any symbol from your list

print(f"--- Top 10 Parameter Combinations for {target_symbol} (Sorted by Avg_10D_Ret) ---")

# Filter the performance_summary we created above
symbol_drill_down = performance_summary[performance_summary['Symbol'] == target_symbol]

# Display the results
display(symbol_drill_down[display_cols].sort_values('Avg_10D_Ret', ascending=False).head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- Top 10 Parameter Combinations for NVDA (Sorted by Avg_10D_Ret) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
1881,NVDA,11,30,29,69.0%,1.15%,69.0%,1.87%,62.1%,5.76%
1878,NVDA,10,30,31,67.7%,1.15%,67.7%,1.78%,61.3%,5.45%
1884,NVDA,12,30,29,65.5%,0.88%,58.6%,1.31%,65.5%,5.03%
1876,NVDA,9,30,37,67.6%,1.42%,62.2%,1.80%,56.8%,5.01%
1890,NVDA,14,30,28,75.0%,2.21%,57.1%,1.53%,57.1%,4.75%
1887,NVDA,13,30,25,76.0%,2.07%,64.0%,2.20%,60.0%,4.67%
1891,NVDA,15,10,9,66.7%,1.43%,44.4%,1.31%,55.6%,4.65%
1883,NVDA,12,20,28,50.0%,0.01%,57.1%,1.27%,75.0%,4.63%
1880,NVDA,11,20,27,59.3%,0.67%,70.4%,1.65%,66.7%,4.56%
1873,NVDA,8,30,39,69.2%,1.73%,61.5%,1.52%,59.0%,4.45%


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Prepare the Aggregate Data (if not already done in Cell 2)
param_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': 'mean',
    'Ret_5D': 'mean',
    'Ret_10D': 'mean'
}).reset_index()

# 2. Define the horizons we want to visualize
horizons = [('3D', 'Ret_3D'), ('5D', 'Ret_5D'), ('10D', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Average Returns) ---")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_table = param_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    # Create the visualization using Pandas Styler for a clean Colab output
    print(f"\n{label} Horizon Average Return:")
    display(pivot_table.style.background_gradient(cmap='RdYlGn', axis=None)
            .format("{:.2%}")
            .set_caption(f"Average {label} Return by SMA and BB Period"))

# Optional: If you prefer a more "visual" graphical heatmap using Seaborn:
# plt.figure(figsize=(10, 6))
# sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap="RdYlGn")
# plt.title("10D Return Surface")
# plt.show()

--- PARAMETER SURFACE HEATMAPS (Average Returns) ---

3D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.47%,0.39%,0.33%
4,0.37%,0.24%,0.20%
5,-0.09%,0.15%,0.13%
6,-0.25%,0.16%,0.22%
7,-0.25%,0.07%,0.19%
8,-0.16%,0.16%,0.31%
9,-0.17%,0.15%,0.31%
10,nan%,0.30%,0.40%
11,0.94%,0.37%,0.48%



5D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.87%,0.33%,0.29%
4,0.91%,0.18%,0.20%
5,-0.13%,0.08%,0.16%
6,-0.39%,0.05%,0.20%
7,-0.28%,0.08%,0.24%
8,-0.31%,0.09%,0.30%
9,-0.12%,0.18%,0.32%
10,nan%,0.32%,0.42%
11,0.77%,0.33%,0.47%



10D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,1.04%,0.97%,0.96%
4,0.89%,0.75%,0.76%
5,0.38%,0.60%,0.69%
6,0.25%,0.79%,0.89%
7,0.59%,0.98%,1.07%
8,0.85%,1.11%,1.22%
9,0.75%,1.13%,1.26%
10,nan%,1.16%,1.32%
11,0.82%,1.29%,1.44%


In [7]:
# 1. Prepare the Win Rate Aggregate Data
# We calculate the mean of the boolean (Return > 0) to get the percentage of wins
win_rate_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': lambda x: (x > 0).mean(),
    'Ret_5D': lambda x: (x > 0).mean(),
    'Ret_10D': lambda x: (x > 0).mean()
}).reset_index()

# 2. Define the horizons for visualization
horizons = [('3-Day', 'Ret_3D'), ('5-Day', 'Ret_5D'), ('10-Day', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---")
print("Focus: Probability of a positive return across the portfolio.")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_win_rate = win_rate_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    print(f"\n{label} Win Rate Surface:")
    # Using a 0.5 (50%) midpoint for the gradient helps highlight 'Edge' vs 'Coin Flip'
    display(pivot_win_rate.style.background_gradient(cmap='RdYlGn', vmin=0.45, vmax=0.65, axis=None)
            .format("{:.1%}")
            .set_caption(f"{label} Win Rate by SMA and BB Period"))

--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---
Focus: Probability of a positive return across the portfolio.

3-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,51.1%,52.7%,52.5%
4,50.1%,52.0%,52.0%
5,48.7%,51.4%,51.5%
6,47.8%,51.5%,52.3%
7,47.5%,51.5%,52.4%
8,47.6%,51.9%,53.5%
9,47.7%,51.7%,53.8%
10,nan%,52.9%,54.8%
11,52.5%,53.2%,55.4%



5-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,50.0%,51.5%,51.6%
4,49.8%,50.8%,51.2%
5,48.2%,49.9%,50.8%
6,48.0%,50.8%,51.6%
7,48.4%,51.2%,51.9%
8,48.7%,52.0%,52.6%
9,49.5%,52.1%,53.0%
10,nan%,53.2%,53.9%
11,51.5%,52.9%,54.1%



10-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,52.8%,53.2%,53.0%
4,51.6%,52.2%,52.3%
5,50.4%,51.5%,51.8%
6,49.9%,52.1%,52.4%
7,50.4%,53.0%,53.1%
8,51.6%,53.7%,53.7%
9,52.0%,54.6%,54.8%
10,nan%,54.9%,55.0%
11,56.5%,56.0%,56.0%


In [8]:
# 1. Create a copy for export to keep the original data intact
export_df = global_strategy_raw.copy()

# 2. Round all numerical columns to 4 decimal places
# This ensures that Win Rates (0.6543 -> 0.65) and Returns (0.0123 -> 0.01) are consistent
export_df = export_df.round(4)

# 3. Rename columns for the final report
export_df = export_df.rename(columns={
    'SMA_Param': 'SMA',
    'BB_Param': 'BB',
    'Total_Trades': 'Trades'
})

# 4. Export to CSV string
# float_format="%.4f" is a safety measure to ensure trailing zeros are kept (e.g., 0.50)
csv_output = export_df.to_csv(index=False, float_format="%.4f")

print("--- COPY AND PASTE THIS TO GEMINI ---")
print(csv_output)

--- COPY AND PASTE THIS TO GEMINI ---
SMA,BB,Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
12,10,307,0.5016,0.0032,0.5505,0.0089,0.5863,0.0171
14,10,398,0.5302,0.0080,0.5427,0.0097,0.5729,0.0215
11,10,200,0.5250,0.0094,0.5150,0.0077,0.5650,0.0082
12,20,1481,0.5321,0.0048,0.5375,0.0050,0.5604,0.0117
11,20,1649,0.5318,0.0037,0.5288,0.0033,0.5603,0.0129
11,30,1839,0.5541,0.0048,0.5405,0.0047,0.5601,0.0144
13,10,372,0.5108,0.0039,0.5296,0.0075,0.5591,0.0149
12,30,1714,0.5543,0.0051,0.5478,0.0056,0.5589,0.0132
14,20,1188,0.5269,0.0053,0.5261,0.0055,0.5547,0.0124
13,20,1335,0.5236,0.0042,0.5296,0.0039,0.5513,0.0108
10,30,1946,0.5483,0.0040,0.5391,0.0042,0.5504,0.0132
13,30,1611,0.5493,0.0048,0.5438,0.0050,0.5500,0.0123
14,30,1536,0.5462,0.0055,0.5391,0.0058,0.5495,0.0127
10,20,1784,0.5286,0.0030,0.5320,0.0032,0.5488,0.0116
9,30,2084,0.5379,0.0031,0.5298,0.0032,0.5485,0.0126
9,20,1957,0.5171,0.0015,0.5212,0.0018,0.5457,0.0113
15,10,425,0.4988,0.0034,0.5294,0.00

In [9]:
# 1. Aggregate Advanced Risk Metrics from Raw Data
# We focus on the 10-Day window as it showed the highest potential for spreads
stress_test = master_trade_log.groupby(['SMA_Param', 'BB_Param'])['Ret_10D'].agg([
    ('Trades', 'count'),
    ('Avg_Ret', 'mean'),
    ('Median_Ret', 'median'),
    ('Std_Dev', 'std'),
    ('Win_Rate', lambda x: (x > 0).mean())
]).reset_index()

# 2. Calculate Expectancy Score (Assuming 1:1 Risk/Reward Ratio)
# Formula: (WinRate * 1) - (LossRate * 1)
# A score of 0.10 means you expect to make $0.10 for every $1.00 risked over time.
stress_test['Expectancy_Score'] = (stress_test['Win_Rate'] * 2) - 1

# 3. Sort by Expectancy Score
stress_test = stress_test.sort_values('Expectancy_Score', ascending=False)

print("--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---")
print("Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.")

display(stress_test.head(10).style.format({
    'Avg_Ret': '{:.2%}',
    'Median_Ret': '{:.2%}',
    'Std_Dev': '{:.2%}',
    'Win_Rate': '{:.1%}',
    'Expectancy_Score': '{:.3f}'
}).background_gradient(subset=['Expectancy_Score'], cmap='Greens'))

--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---
Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.


,SMA_Param,BB_Param,Trades,Avg_Ret,Median_Ret,Std_Dev,Win_Rate,Expectancy_Score
26,12,10,304,1.71%,1.15%,10.83%,58.6%,0.173
32,14,10,393,2.15%,0.88%,10.88%,57.3%,0.146
23,11,10,199,0.82%,0.93%,9.81%,56.5%,0.130
27,12,20,1475,1.17%,0.91%,11.17%,56.0%,0.121
24,11,20,1641,1.29%,0.87%,11.60%,56.0%,0.121
25,11,30,1828,1.44%,0.96%,11.64%,56.0%,0.120
29,13,10,368,1.49%,0.80%,10.97%,55.9%,0.118
28,12,30,1706,1.32%,0.97%,11.16%,55.9%,0.118
33,14,20,1183,1.24%,0.89%,11.36%,55.5%,0.109
30,13,20,1330,1.08%,0.87%,11.48%,55.1%,0.103
